In [1]:
%matplotlib inline
from effective_quadratures.parameter import Parameter
from effective_quadratures.polynomial import Polynomial
from effective_quadratures.indexset import IndexSet
from effective_quadratures.effectivequads import EffectiveSubsampling
from effective_quadratures.computestats import Statistics
import numpy as np

<h1> Vegetation UQ with Effective-Quadratures

Consider a uniform variation in the following parameters:
    
$$\begin{array}{lll} \hline
Variable & Range & Description \\ \hline
\rho_{shoot} & [38.2, 250.4] & Shoot \; density \; (/m^2)\\ 
L & [0.157, 0.313] & Plant \; length \; (m)\\ 
d & [0.002, 0.01] & Diameter \;  (m)\\ 
t & [0.0002, 0.001] & Thickness \; (m)\\  \hline
\end{array}$$

Being by setting up the ranges for the four parameters

In [2]:
x1 = Parameter(lower=38.2, upper=250.4, points=3)
x2 = Parameter(lower=0.157, upper=0.313, points=3)
x3 = Parameter(lower=0.002, upper=0.01,  points=3)
x4 = Parameter(lower=0.0002, upper=0.001, points=3)
parameters = [x1, x2, x3, x4]

Set the polynomial basis; this will dictate how many function evaluations we will require. Bear in mind that a quadratic in 4 dimensions requires $3^4=81$ points. To reduce the cost we will opt for a total order basis.

In [3]:
orders = [2, 2, 2, 2]
polybasis = IndexSet("total order", orders)
maximum_number_of_evals = polybasis.getCardinality()

We print out the design-of-experiment required to compute the polynomial model

In [4]:
esq = EffectiveSubsampling(parameters, polybasis)
points = esq.getPointsToEvaluate(maximum_number_of_evals)
print points

[[  1.44300000e+02   2.35000000e-01   6.00000000e-03   6.00000000e-04]
 [  6.21152934e+01   1.74581460e-01   2.90161332e-03   6.00000000e-04]
 [  2.26484707e+02   2.95418540e-01   6.00000000e-03   2.90161332e-04]
 [  1.44300000e+02   1.74581460e-01   9.09838668e-03   9.09838668e-04]
 [  1.44300000e+02   2.95418540e-01   2.90161332e-03   9.09838668e-04]
 [  6.21152934e+01   2.95418540e-01   6.00000000e-03   2.90161332e-04]
 [  2.26484707e+02   1.74581460e-01   6.00000000e-03   2.90161332e-04]
 [  1.44300000e+02   2.95418540e-01   9.09838668e-03   9.09838668e-04]
 [  1.44300000e+02   2.35000000e-01   9.09838668e-03   2.90161332e-04]
 [  2.26484707e+02   2.35000000e-01   6.00000000e-03   9.09838668e-04]
 [  1.44300000e+02   2.35000000e-01   2.90161332e-03   2.90161332e-04]
 [  6.21152934e+01   2.35000000e-01   6.00000000e-03   9.09838668e-04]
 [  6.21152934e+01   1.74581460e-01   6.00000000e-03   2.90161332e-04]
 [  6.21152934e+01   2.35000000e-01   9.09838668e-03   6.00000000e-04]
 [  1.

Taking the output from the text file for dissipation, we have:

In [9]:
Output = [15.9881,16.5091,16.0162,15.9950,16.0310,16.4592,16.0958,15.8507,16.0757,15.9252,16.4301,16.1259,16.4682,16.0501,16.2200]
Output = np.mat(Output)

Now we solve the least squares problem, using the esq object

In [10]:
x = esq.solveLeastSquares(maximum_number_of_evals, Output.T)

Having solved for the coefficients for the polynomial expansion, we can now compute statistics!

In [11]:
vegeUQ = Statistics(x, polybasis)
mean = vegeUQ.getMean()
variance = vegeUQ.getVariance()
sobol = vegeUQ.getFirstOrderSobol()

Yay! We are done. Now let's print out the outputs.

In [12]:
print mean, variance
print sobol

16.1055166667 0.039443688708
[ 0.34801859  0.03922184  0.29234309  0.24367443]
